In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from sklearn.datasets import make_moons
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms

from tqdm import tqdm

# Exemple le plus simple possible sur un dataset généré avec le plus petit modèle possible pour tester les gradients

In [2]:
# Modèle simple de classification
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Fonction pour mettre à jour les paramètres du modèle
def set_model_params(model, theta):
    with torch.no_grad():
        for param, new_param in zip(model.parameters(), theta):
            param.copy_(new_param)

# Fonction de log-probabilité avec prior gaussien
def log_prob_func(model, data, target, prior_std=1.0):
    logits = model(data)
    log_likelihood = -F.cross_entropy(logits, target, reduction='sum')
    prior = -0.5 * sum(torch.sum(p ** 2) for p in model.parameters()) / (prior_std ** 2)
    return log_likelihood + prior

# Fonction pour collecter les gradients
def compute_gradients(model, data, target):
    log_prob = log_prob_func(model, data, target)
    grads = torch.autograd.grad(log_prob, model.parameters(), create_graph=True)
    return grads

# Implémentation de Leapfrog
def leapfrog(theta, r, step_size, num_steps, model, data, target):
    theta = [p.clone().detach().requires_grad_(True) for p in theta]
    r = [ri.clone().detach() for ri in r]

    set_model_params(model, theta)
    grad = compute_gradients(model, data, target)

    for i in range(len(r)):
        r[i] = r[i] + 0.5 * step_size * grad[i]

    for _ in range(num_steps):
        for i in range(len(theta)):
            theta[i] = theta[i] + step_size * r[i]

        set_model_params(model, theta)
        grad = compute_gradients(model, data, target)

        for i in range(len(r)):
            r[i] = r[i] + step_size * grad[i]

    set_model_params(model, theta)
    grad = compute_gradients(model, data, target)

    for i in range(len(r)):
        r[i] = r[i] - 0.5 * step_size * grad[i]

    return theta, r

# Fonction d'acceptation Metropolis-Hastings
def acceptance(theta, r, new_theta, new_r, model, data, target):
    set_model_params(model, theta)
    current_H = -log_prob_func(model, data, target) + 0.5 * sum(torch.sum(ri**2) for ri in r)
    set_model_params(model, new_theta)
    proposed_H = -log_prob_func(model, data, target) + 0.5 * sum(torch.sum(ri**2) for ri in new_r)

    accept_prob = torch.exp(current_H - proposed_H)
    if torch.rand(1) < accept_prob:
        return new_theta, 1
    else:
        return theta, 0

In [3]:
# Hyperparamètres
input_dim = 2
hidden_dim = 16
output_dim = 2
num_samples = 100
step_size = 0.01
num_steps = 10
batch_size = 32

# Chargement des données (Two Moons)
X, y = make_moons(n_samples=1000, noise=0.1)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialisation du modèle
model = SimpleNN(input_dim, hidden_dim, output_dim)
theta = [p.clone().detach() for p in model.parameters()]

# Échantillonnage HMC
samples = []
ratio_acceptations = 0
for _ in range(num_samples):
    r = [torch.randn_like(p) for p in theta]
    data, target = next(iter(dataloader))
    new_theta, new_r = leapfrog(theta, r, step_size, num_steps, model, data, target)
    theta, acceptation = acceptance(theta, r, new_theta, new_r, model, data, target)
    ratio_acceptations += acceptation
    samples.append([p.clone().detach() for p in theta])

print(f"Échantillonnage terminé avec {len(samples)} échantillons et un ratio de {ratio_acceptations/num_samples}.")


Échantillonnage terminé avec 100 échantillons et un ratio de 0.99.


In [4]:
# Charger le dataset de test
X_test, y_test = make_moons(n_samples=300, noise=0.1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Initialiser les prédictions accumulées
predictions = torch.zeros((len(samples), X_test.shape[0], output_dim))

# Effectuer des prédictions avec chaque échantillon
for i, sample in enumerate(samples):
    set_model_params(model, sample)  # Charger les paramètres échantillonnés
    logits = model(X_test)  # Prédictions sur le test set
    predictions[i] = F.softmax(logits, dim=1)  # Convertir en probabilités

# Moyenne des prédictions (Bayesian Model Averaging)
avg_predictions = predictions.mean(dim=0)  # Moyenne sur tous les échantillons

# Prédictions finales (classe avec probabilité max)
y_pred = avg_predictions.argmax(dim=1)

# Calcul de l'accuracy
accuracy = (y_pred == y_test).float().mean().item()
print(f"Accuracy (BMA): {accuracy:.4f}")


Accuracy (BMA): 0.8800


On obtient 90% d'accuracy

# Test sur MNIST avec des CNN (plus dur)

In [5]:
# Modèle de classification avec plus de couches
class DeepNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)


class CNN(nn.Module):
    def __init__(self, output_dim=10):
        super(CNN, self).__init__()

        # Définir les couches convolutives
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)  # Images MNIST (1 canal)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        #self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        # Couches fully-connected
        self.fc1 = nn.Linear(64 * 7 * 7, 512)  # Taille après réduction par les convolutions (28x28 -> 3x3 après convolutions et pooling)
        self.fc2 = nn.Linear(512, output_dim)  # 10 classes pour MNIST

        # Couches de pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Conv1 + ReLU + Pooling
        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pooling
        #x = self.pool(F.relu(self.conv3(x)))  # Conv3 + ReLU + Pooling
        x = x.view(-1, 64 * 7 * 7)  # Aplatir les données pour la couche fully-connected
        x = F.relu(self.fc1(x))  # Fully connected 1
        x = self.fc2(x)  # Fully connected 2 (sortie)
        return x


# Fonction pour mettre à jour les paramètres du modèle
def set_model_params(model, theta):
    with torch.no_grad():
        for param, new_param in zip(model.parameters(), theta):
            param.copy_(new_param)

# Fonction de log-probabilité avec prior gaussien
def log_prob_func(model, data, target, prior_std=1.0):
    logits = model(data)
    log_likelihood = -F.cross_entropy(logits, target, reduction='sum')
    prior = -0.5 * sum(torch.sum(p ** 2) for p in model.parameters()) / (prior_std ** 2)
    return log_likelihood + prior

# Fonction pour collecter les gradients
def compute_gradients(model, data, target):
    log_prob = log_prob_func(model, data, target)
    grads = torch.autograd.grad(log_prob, model.parameters(), create_graph=True)
    return grads

# Implémentation de Leapfrog
def leapfrog(theta, r, step_size, num_steps, model, data, target):
    theta = [p.clone().detach().requires_grad_(True) for p in theta]
    r = [ri.clone().detach() for ri in r]

    set_model_params(model, theta)
    grad = compute_gradients(model, data, target)

    for i in range(len(r)):
        r[i] = r[i] + 0.5 * step_size * grad[i]

    for _ in range(num_steps):
        for i in range(len(theta)):
            theta[i] = theta[i] + step_size * r[i]

        set_model_params(model, theta)
        grad = compute_gradients(model, data, target)

        for i in range(len(r)):
            r[i] = r[i] + step_size * grad[i]

    set_model_params(model, theta)
    grad = compute_gradients(model, data, target)

    for i in range(len(r)):
        r[i] = r[i] - 0.5 * step_size * grad[i]

    return theta, r

# Fonction d'acceptation Metropolis-Hastings
def acceptance(theta, r, new_theta, new_r, model, data, target, device):
    set_model_params(model, theta)
    current_H = -log_prob_func(model, data, target) + 0.5 * sum(torch.sum(ri**2) for ri in r)

    set_model_params(model, new_theta)
    proposed_H = -log_prob_func(model, data, target) + 0.5 * sum(torch.sum(ri**2) for ri in new_r)

    accept_prob = torch.exp(current_H - proposed_H)

    if torch.rand(1).to(device) < accept_prob:
        #print(accept_prob)
        return new_theta, 1
    else:
        return theta, 0

def HMC_update(trajectory_length, N_burnin, model, trainloader, delta, num_sample, device, N_leapfrog=None):
    theta = [p.clone().detach().to(device) for p in model.parameters()]  # Déplacer les paramètres du modèle sur le GPU
    if N_leapfrog is None:
      N_leapfrog = int(trajectory_length // delta)

    if N_leapfrog > 240:
        N_leapfrog = 240 # With N_leapfrog too high, the memory is saturated
    print(f"Number of leapfrog steps : {N_leapfrog}")

    # Burn in phase
    for _ in tqdm(range(N_burnin), desc="Burn in phase"):
      data, target = next(iter(trainloader))
      data, target = data.to(device), target.to(device)  # Envoyer les données sur le GPU
      r = [torch.randn_like(p).to(device) for p in theta]
      new_theta, _ = leapfrog(theta, r, delta, N_leapfrog, model, data, target)
      theta = new_theta

      # Reduce memory cost
      del r, new_theta
      torch.cuda.empty_cache()
    # Échantillonnage HMC
    samples = []
    ratio_acceptations = 0
    for _ in tqdm(range(num_sample), desc="HMC sampling"):
        r = [torch.randn_like(p).to(device) for p in theta]  # Initialisation de r sur GPU
        data, target = next(iter(trainloader))
        data, target = data.to(device), target.to(device)  # Envoyer les données sur le GPU

        new_theta, new_r = leapfrog(theta, r, delta, N_leapfrog, model, data, target)
        theta, acceptation = acceptance(theta, r, new_theta, new_r, model, data, target, device)
        ratio_acceptations += acceptation

        # Delete the cache and variable to reduce the memory cost
        del r, new_theta, new_r
        torch.cuda.empty_cache()

        samples.append([p.clone().detach().cpu() for p in theta])
    print(f"Échantillonnage terminé avec {len(samples)} échantillons et un ratio de {ratio_acceptations/num_sample}.")
    return samples, ratio_acceptations/num_sample

### Génération des samples

In [6]:
# Vérifier si un GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparamètres
output_dim = 10  # MNIST a 10 classes
num_samples = 200
step_size = 0.001
num_steps = 10
batch_size = 64

# Chargement des données MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])  # MNIST est en niveaux de gris

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Initialisation du modèle et envoi du modèle sur le GPU
model = CNN(output_dim).to(device) # Envoyer le modèle sur le GPU
theta = [p.clone().detach().to(device) for p in model.parameters()]  # Déplacer les paramètres du modèle sur le GPU

# Parameters for the HMC
tau_hat = torch.pi*torch.std(trainset.data.float() / 255)/2 # Trajectory length estimated in the paper
delta = 1e-3
N_burnin = 10
N_leapfrog = 100

# samples, ratio = HMC_update(tau_hat, N_burnin, model, trainloader, delta, num_samples, device, N_leapfrog)



In [7]:

# # Échantillonnage HMC
# samples = []
# ratio_acceptations = 0
# for _ in range(num_samples):
#     r = [torch.randn_like(p).to(device) for p in theta]  # Initialisation de r sur GPU
#     data, target = next(iter(trainloader))
#     data, target = data.to(device), target.to(device)  # Envoyer les données sur le GPU

#     new_theta, new_r = leapfrog(theta, r, step_size, num_steps, model, data, target)
#     theta, acceptation = acceptance(theta, r, new_theta, new_r, model, data, target, device)
#     ratio_acceptations += acceptation
#     samples.append([p.clone().detach().to(device) for p in theta])  # Stocker sur GPU

# print(f"Échantillonnage terminé avec {len(samples)} échantillons et un ratio de {ratio_acceptations/num_samples}.")

### Calcul de l'accuracy

In [8]:
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, drop_last=True) # drop_last to stack the outputs and target later

# Fonction d'inférence avec la méthode BMA
def bma_inference(model, samples, data, device):
    # On suppose que `samples` est une liste des paramètres du modèle (après HMC)
    # On va faire une prédiction pour chaque échantillon
    preds = []
    for sample in samples:
        # Charger les paramètres dans le modèle
        for param, s in zip(model.parameters(), sample):
            param.data = s.data  # Mettre à jour les paramètres du modèle avec l'échantillon

        model.to(device)  # Mettre le modèle sur le GPU
        # Prédire avec le modèle actuel
        output = model(data)
        preds.append(F.softmax(output, dim=1))  # Utilisation de softmax pour obtenir les probabilités

    # Calcul de la moyenne des prédictions
    avg_preds = torch.stack(preds).mean(0)  # Moyenne sur tous les échantillons (axis 0)

    # Retourner la classe avec la plus haute probabilité
    return avg_preds, avg_preds.argmax(dim=1)

# # Calcul de l'accuracy sur le testset
# model.eval()  # Mettre le modèle en mode évaluation
# correct = 0
# total = 0
# with torch.no_grad():
#     for data, target in tqdm(testloader, desc="Predicting the test set"):
#         data, target = data.to(device), target.to(device)  # Envoyer les données sur le GPU

#         # Obtenez les prédictions avec BMA
#         _, preds = bma_inference(model, samples, data, device)

#         # Compter les prédictions correctes
#         correct += (preds == target).sum().item()
#         total += target.size(0)

# accuracy = correct / total
# print(f"Accuracy sur le testset : {accuracy * 100:.2f}%")

### 75% d'accuracy enfinnnnnnnnnnnnnnnnn


## Tests on the parameters

Function to compute all the metrics :
- The Expected Calibration Error (ECE)
- The accuracy
- The confidence

In [9]:
def compute_calibration_curve(outputs, labels, num_bins=20):
    """
    Compute the calibration curve and Expected Calibration Error (ECE).

    Parameters:
        outputs (torch.Tensor): Model outputs (probabilities), shape (N, num_classes).
        labels (torch.Tensor): Ground-truth labels, shape (N,).
        num_bins (int): Number of bins for calibration.

    Returns:
        dict: Contains "confidence", "accuracy", "proportions", and "ece".
    """
    # Flatten outputs and labels
    labels = labels.view(-1)  # Ensure labels are 1D
    outputs = outputs.view(labels.size(0), -1)  # Ensure correct shape

    # Get confidences (max probabilities) and predicted classes
    confidences, predictions = torch.max(outputs, dim=1)
    accuracies = (predictions == labels).float()  # Boolean to float (1=correct, 0=incorrect)

    # Sort confidences to determine bin edges
    num_inputs = confidences.shape[0]
    bins = torch.sort(confidences)[0][::(num_inputs + num_bins - 1) // num_bins]
    if num_inputs % ((num_inputs + num_bins - 1) // num_bins) != 1:
        bins = torch.cat((bins, confidences.max().unsqueeze(0)))

    # Initialize bin statistics
    bin_confidences, bin_accuracies, bin_proportions = [], [], []
    ece = 0.0  # Expected Calibration Error

    # Compute calibration metrics for each bin
    for bin_lower, bin_upper in zip(bins[:-1], bins[1:]):
        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        prop_in_bin = in_bin.float().mean().item()

        if prop_in_bin > 0:
            accuracy_in_bin = accuracies[in_bin].mean().item()
            avg_confidence_in_bin = confidences[in_bin].mean().item()
            ece += abs(avg_confidence_in_bin - accuracy_in_bin) * prop_in_bin

            bin_confidences.append(avg_confidence_in_bin)
            bin_accuracies.append(accuracy_in_bin)
            bin_proportions.append(prop_in_bin)

    # Convert lists to tensors
    return {
        "confidence": torch.tensor(bin_confidences),
        "accuracy": torch.tensor(bin_accuracies),
        "proportions": torch.tensor(bin_proportions),
        "ece": ece
    }


Now that we ensure the model works, we will reproduce the experience on the trajectory length and hmc chain


In [10]:
def hmc_test(trajectory_length, N_burnin, model, trainloader, delta, num_sample, testloader, samples, device, N_leapfrog=None, num_classes=10):
  model.to(device)
  # HMC update
  samples, ratio = HMC_update(trajectory_length, N_burnin, model, trainloader, delta, num_sample, device, N_leapfrog)

  outputs = []
  targets = []
  model.eval()  # Mettre le modèle en mode évaluation
  with torch.no_grad():
      for data, target in tqdm(testloader, desc="Predicting the test set"):
          data, target = data.to(device), target.to(device)  # Envoyer les données sur le GPU

          # Obtenez les prédictions avec BMA
          preds, avg_pred = bma_inference(model, samples, data, device)

          outputs.append(preds)
          targets.append(target)
  outputs = torch.stack(outputs).view(-1, num_classes)
  targets = torch.stack(targets).view(-1)

  results = compute_calibration_curve(outputs, targets)

  print("\nAccuracy : ", results["accuracy"].mean().item())
  print("Confidence : ", results["confidence"].mean().item())
  print("ECE : ", results["ece"])

  log_likelihood = -F.cross_entropy(outputs, targets, reduction='sum')
  print("log likelihood : ", log_likelihood.item())
  return results


In [11]:
tau_hat = torch.pi*torch.std(trainset.data.float() / 255)/2 # Trajectory length estimated in the paper
delta = 1e-3
N_burnin = 10
N_leapfrog = 100

prop_tau_test = [0.25, 0.5, 1, 1.25, 1.5]
for prop in prop_tau_test:
  tau_test = prop*tau_hat
  print(f"-------------- Trajectory length : {tau_test} -------------------")
  results = hmc_test(tau_test, N_burnin, model, trainloader, delta, num_samples, testloader, samples, device)


-------------- Trajectory length : 0.12099365890026093 -------------------
Number of leapfrog steps : 120


HMC sampling: 100%|██████████| 200/200 [01:43<00:00,  1.94it/s]


Échantillonnage terminé avec 200 échantillons et un ratio de 0.455.


Predicting the test set: 100%|██████████| 156/156 [01:11<00:00,  2.19it/s]



Accuracy :  0.9184038043022156
Confidence :  0.4635261595249176
ECE :  0.4550315805553581
log likelihood :  -19554.802734375
-------------- Trajectory length : 0.24198731780052185 -------------------
Number of leapfrog steps : 240


...


Accuracy :  0.12203727662563324
Confidence :  0.1267998218536377
ECE :  0.03300230029812401
log likelihood :  -22976.17578125
-------------- Trajectory length : 0.7259619235992432 -------------------
Number of leapfrog steps : 240


HMC sampling: 100%|██████████| 200/200 [03:31<00:00,  1.06s/it]


Échantillonnage terminé avec 200 échantillons et un ratio de 0.555.


Predicting the test set: 100%|██████████| 156/156 [01:11<00:00,  2.19it/s]


Accuracy :  0.12677502632141113
Confidence :  0.12262680381536484
ECE :  0.03595462811980893
log likelihood :  -22980.140625
